In [ ]:
import pandas as pd
import numpy as np
import plotly.graph_objects as go
import plotly.io as pio
import json

# Set plotly to use CDN for resources
pio.templates.default = "plotly_white"

# Load the Excel file
file_path = '../../data/raw_data/cali_hiv_aids/persons-living-with-hiv-aids.xlsx'
df = pd.read_excel(file_path)

# Filter rows where Category contains "Transmission Category:"
df = df[df['Category'].str.contains('Transmission Category:', na=False)]

# Function to extract population type from Category
def extract_population(category):
    if 'Male Adult' in category:
        return 'Male'
    elif 'Female Adult' in category:
        return 'Female'
    elif 'Child' in category:
        return 'Child'
    else:
        return 'Unknown'

# Add a Population column
df['Population'] = df['Category'].apply(extract_population)

# Make sure Year is integer type and Count is numeric
df['Year'] = df['Year'].astype(int)
df['Count'] = pd.to_numeric(df['Count'], errors='coerce')

# Group by Year, Population, and Group to get totals
grouped_df = df.groupby(['Year', 'Population', 'Group'])['Count'].sum().reset_index()

# Convert numpy data types to native Python types for JSON serialization
grouped_df['Year'] = grouped_df['Year'].astype(int)
grouped_df['Count'] = grouped_df['Count'].astype(float)

# Define a function to prepare data for a specific population filter
def prepare_data_for_population(population_filter='All'):
    if population_filter == 'All':
        # For "All", group by Year and Group, summing Counts
        filtered_df = grouped_df.groupby(['Year', 'Group'])['Count'].sum().reset_index()
    else:
        # Filter by the selected population
        filtered_df = grouped_df[grouped_df['Population'] == population_filter]
    
    # Convert numpy data types to native Python types
    filtered_df['Year'] = filtered_df['Year'].astype(int)
    filtered_df['Count'] = filtered_df['Count'].astype(float)
    
    # Prepare data for JSON
    years = sorted([int(y) for y in filtered_df['Year'].unique()])
    groups = sorted(filtered_df['Group'].unique().tolist())
    
    # Create a nested dictionary
    data_dict = {}
    for year in years:
        data_dict[str(year)] = {}
        for group in groups:
            data_dict[str(year)][group] = 0
    
    # Fill in the data
    for _, row in filtered_df.iterrows():
        year = str(int(row['Year']))
        group = row['Group']
        count = float(row['Count'])
        data_dict[year][group] = count
    
    return {
        'years': years,
        'groups': groups,
        'data': data_dict
    }

# Prepare data for all population filters
all_data = {
    'All': prepare_data_for_population('All'),
    'Male': prepare_data_for_population('Male'),
    'Female': prepare_data_for_population('Female'),
    'Child': prepare_data_for_population('Child')
}

# Create colors for the groups
groups_across_all = set()
for pop_data in all_data.values():
    groups_across_all.update(pop_data['groups'])

colors = {
    'Male-to-male sexual contact (MMSC)': '#1f77b4',
    'Injection drug use (IDU)': '#ff7f0e',
    'MMSC and IDU': '#2ca02c',
    'High-risk heterosexual contact (HRH)': '#d62728',
    'Heterosexual contact (Non-HRH)': '#9467bd',
    'Perinatal': '#8c564b',
    'Unknown risk': '#e377c2',
    'Other': '#7f7f7f'
}

# For any groups not in our predefined colors, assign a random color
for group in groups_across_all:
    if group not in colors:
        colors[group] = f'#{hash(group) % 0xffffff:06x}'

# Class to handle NumPy types in JSON serialization
class NumpyEncoder(json.JSONEncoder):
    def default(self, obj):
        if isinstance(obj, (np.integer, np.int64)):
            return int(obj)
        elif isinstance(obj, (np.floating, np.float64)):
            return float(obj)
        elif isinstance(obj, np.ndarray):
            return obj.tolist()
        return json.JSONEncoder.default(self, obj)

# Convert the data to JSON for embedding in the HTML
json_data = json.dumps(all_data, cls=NumpyEncoder)
json_colors = json.dumps(colors)

# Create the HTML file with everything embedded
html_content = f'''
<!DOCTYPE html>
<html>
<head>
    <title>HIV/AIDS Transmission Categories in California (2011-2017)</title>
    <script src="https://cdn.plot.ly/plotly-latest.min.js"></script>
    <style>
        body {{
            font-family: Arial, sans-serif;
            margin: 0;
            padding: 20px;
            max-width: 1200px;
            margin: 0 auto;
        }}
        h1 {{
            margin-bottom: 20px;
        }}
        .description {{
            margin-bottom: 20px;
        }}
        .filter-container {{
            margin-bottom: 20px;
        }}
        label {{
            font-weight: bold;
            margin-right: 10px;
        }}
        select {{
            padding: 5px;
            min-width: 150px;
        }}
        #chart-container {{
            width: 100%;
            height: 500px;
        }}
        .key-insights {{
            margin-top: 30px;
        }}
        .key-insights h2 {{
            margin-bottom: 10px;
        }}
        .key-insights ul {{
            padding-left: 20px;
        }}
        .key-insights li {{
            margin-bottom: 8px;
        }}
        .data-source {{
            margin-top: 30px;
            font-size: 0.8em;
            color: #666;
        }}
    </style>
</head>
<body>
    <h1>HIV/AIDS Transmission Categories in California (2011-2017)</h1>
    
    <div class="description">
        <p>This visualization shows the number of people living with HIV/AIDS by transmission category over time.
        The data demonstrates that HIV affects various populations, not just men who have sex with men.</p>
    </div>
    
    <div class="filter-container">
        <label for="population-filter">Filter by Population:</label>
        <select id="population-filter" onchange="updateChart(this.value)">
            <option value="All">All Populations</option>
            <option value="Male">Male Adult or Adolescent</option>
            <option value="Female">Female Adult or Adolescent</option>
            <option value="Child">Child (&lt;12 Years)</option>
        </select>
    </div>
    
    <div id="chart-container"></div>
    
    <div class="key-insights">
        <h2>Key Insights:</h2>
        <ul>
            <li>Male-to-male sexual contact (MMSC) represents a significant portion of cases, but HIV affects many other populations as well</li>
            <li>High-risk heterosexual contact is a major transmission category, especially among women</li>
            <li>Injection drug use remains an important risk factor across populations</li>
            <li>Perinatal transmission (mother-to-child) has decreased over time but still occurs</li>
        </ul>
    </div>
    
    <div class="data-source">
        <p>Data source: California Department of Public Health</p>
    </div>
    
    <script>
        // Store the data embedded in the HTML
        const allData = {json_data};
        const colors = {json_colors};
        
        function updateChart(population) {{
            // Get the data for this population
            const popData = allData[population];
            const years = popData.years;
            const groups = popData.groups;
            const data = popData.data;
            
            // Create the traces for each group
            const traces = [];
            
            for (const group of groups) {{
                const x = [];
                const y = [];
                
                for (const year of years) {{
                    x.push(year);
                    y.push(data[String(year)][group]);
                }}
                
                traces.push({{
                    x: x,
                    y: y,
                    type: 'scatter',
                    mode: 'lines+markers',
                    name: group,
                    line: {{
                        color: colors[group],
                        width: 2
                    }},
                    marker: {{
                        size: 8
                    }}
                }});
            }}
            
            // Create the layout
            const layout = {{
                title: `HIV/AIDS Transmission Categories - ${{population}} Population (2011-2017)`,
                xaxis: {{
                    title: 'Year',
                    tickvals: years
                }},
                yaxis: {{
                    title: 'Number of Cases'
                }},
                legend: {{
                    title: {{
                        text: 'Transmission Category'
                    }}
                }},
                hovermode: 'closest',
                autosize: true
            }};
            
            // Create the plot
            Plotly.newPlot('chart-container', traces, layout);
        }}
        
        // Initialize with "All" data
        document.addEventListener('DOMContentLoaded', function() {{
            updateChart('All');
        }});
    </script>
</body>
</html>
'''

# Write the HTML file
with open('../../interactive_viz_outputs/hiv_visualization.html', 'w', encoding='utf-8') as f:
    f.write(html_content)

print("Interactive visualization created as 'hiv_visualization.html'")
print("You can open this file in any web browser")

Interactive visualization created as 'hiv_visualization.html'
You can open this file in any web browser


/var/folders/vs/3j3cdp2s1nn0lr6x50_9ycj00000gn/T/ipykernel_60230/4211463963.py:52: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/var/folders/vs/3j3cdp2s1nn0lr6x50_9ycj00000gn/T/ipykernel_60230/4211463963.py:53: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/var/folders/vs/3j3cdp2s1nn0lr6x50_9ycj00000gn/T/ipykernel_60230/4211463963.py:52: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in th

In [11]:
import pandas as pd
import numpy as np
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots
import math
import json

# Load the Excel file
file_path = '../../data/raw_data/cali_hiv_aids/persons-living-with-hiv-aids.xlsx'
df = pd.read_excel(file_path)

# Filter rows where Category contains "Transmission Category:"
df = df[df['Category'].str.contains('Transmission Category:', na=False)]

# Function to extract population type from Category
def extract_population(category):
    if 'Male Adult' in category:
        return 'Male'
    elif 'Female Adult' in category:
        return 'Female'
    elif 'Child' in category:
        return 'Child'
    else:
        return 'Unknown'

# Add a Population column
df['Population'] = df['Category'].apply(extract_population)

# Make sure Year is integer type and Count is numeric
df['Year'] = df['Year'].astype(int)
df['Count'] = pd.to_numeric(df['Count'], errors='coerce')

# Group by Year, Population, and Group to get totals
grouped_df = df.groupby(['Year', 'Population', 'Group'])['Count'].sum().reset_index()

# Prepare data for radial visualization
years = sorted(grouped_df['Year'].unique())
populations = sorted(grouped_df['Population'].unique())
all_groups = sorted(grouped_df['Group'].unique())

# Calculate maximum count for normalization
max_count = grouped_df['Count'].max()

# Define a vibrant color scale
colorscale = px.colors.diverging.Spectral

# Create HTML file with plotly.js and visualization
html_content = """
<!DOCTYPE html>
<html>
<head>
    <title>Radial Visualization: HIV Transmission Categories (2011-2017)</title>
    <script src="https://cdn.plot.ly/plotly-latest.min.js"></script>
    <script src="https://d3js.org/d3.v7.min.js"></script>
    <style>
        body {
            font-family: Arial, sans-serif;
            margin: 0;
            padding: 20px;
            max-width: 1200px;
            margin: 0 auto;
        }
        h1 {
            text-align: center;
            margin-bottom: 20px;
        }
        .description {
            text-align: center;
            margin-bottom: 30px;
            max-width: 800px;
            margin-left: auto;
            margin-right: auto;
        }
        .controls {
            display: flex;
            justify-content: center;
            margin-bottom: 20px;
            gap: 20px;
        }
        .control-group {
            display: flex;
            flex-direction: column;
            align-items: center;
        }
        label {
            font-weight: bold;
            margin-bottom: 5px;
        }
        select {
            padding: 5px;
            min-width: 150px;
        }
        #visualization {
            width: 100%;
            height: 800px;
        }
        .key-insights {
            margin-top: 40px;
        }
        .key-insights h2 {
            text-align: center;
        }
        .button {
            background-color: #4CAF50;
            border: none;
            color: white;
            padding: 10px 20px;
            text-align: center;
            text-decoration: none;
            display: inline-block;
            font-size: 16px;
            margin: 4px 2px;
            cursor: pointer;
            border-radius: 4px;
        }
        .button:hover {
            background-color: #45a049;
        }
        .button:disabled {
            background-color: #cccccc;
            cursor: not-allowed;
        }
        .legend {
            margin-top: 20px;
            display: flex;
            justify-content: center;
            flex-wrap: wrap;
            gap: 10px;
        }
        .legend-item {
            display: flex;
            align-items: center;
            margin-right: 15px;
        }
        .legend-color {
            width: 20px;
            height: 20px;
            margin-right: 5px;
            border-radius: 3px;
        }
        .tooltip {
            position: absolute;
            background-color: rgba(255, 255, 255, 0.9);
            border: 1px solid #ddd;
            border-radius: 4px;
            padding: 10px;
            pointer-events: none;
            max-width: 300px;
            z-index: 10;
            box-shadow: 2px 2px 6px rgba(0, 0, 0, 0.2);
        }
    </style>
</head>
<body>
    <h1>HIV Transmission Categories: Beyond a Single Narrative (2011-2017)</h1>
    
    <div class="description">
        <p>This radial visualization shows HIV transmission categories across different populations from 2011-2017. 
        Transmission categories are arranged around the circle, with years as concentric rings moving outward.
        The size and color intensity of each segment represents the number of cases, revealing the diverse
        populations affected by HIV beyond the common misconception that it's "just a gay disease".</p>
    </div>
    
    <div class="controls">
        <div class="control-group">
            <label for="population-filter">Population:</label>
            <select id="population-filter">
                <option value="All">All Populations</option>
                <option value="Male">Male</option>
                <option value="Female">Female</option>
                <option value="Child">Child</option>
            </select>
        </div>
        
        <div class="control-group">
            <label for="view-mode">View Mode:</label>
            <select id="view-mode">
                <option value="radial">Radial View</option>
                <option value="comparative">Comparative View</option>
            </select>
        </div>
        
        <div class="control-group">
            <label for="animation">Animation:</label>
            <button id="play-button" class="button">Play Animation</button>
        </div>
    </div>
    
    <div id="visualization"></div>
    
    <div class="legend" id="legend-container"></div>
    
    <div class="key-insights">
        <h2>Key Insights</h2>
        <ul>
            <li>While male-to-male sexual contact represents a significant number of cases, the visualization reveals substantial HIV transmission through other routes</li>
            <li>High-risk heterosexual contact is a major transmission category, especially among women</li>
            <li>Injection drug use affects both men and women, crossing demographic boundaries</li>
            <li>Perinatal transmission (mother-to-child) exists across the time period, though it has decreased</li>
            <li>The data shows HIV is not confined to any single population group; it affects diverse communities through various transmission routes</li>
        </ul>
    </div>
    
    <script>
    // Data processing and visualization will be inserted here
    
    // Data for the visualization
    const data = PLACEHOLDER_DATA;
    
    // Get DOM elements
    const visualizationContainer = document.getElementById('visualization');
    const populationFilter = document.getElementById('population-filter');
    const viewMode = document.getElementById('view-mode');
    const playButton = document.getElementById('play-button');
    const legendContainer = document.getElementById('legend-container');
    
    // Global variables
    let currentYear = 2011;
    let animationInterval;
    let currentPopulation = 'All';
    let currentViewMode = 'radial';
    let tooltip;
    
    // Initialize tooltip
    function createTooltip() {
        tooltip = document.createElement('div');
        tooltip.className = 'tooltip';
        tooltip.style.display = 'none';
        document.body.appendChild(tooltip);
    }
    
    // Create the color mapping for groups
    const groupColors = {
        'Male-to-male sexual contact (MMSC)': '#1f77b4',
        'Injection drug use (IDU)': '#ff7f0e',
        'MMSC and IDU': '#2ca02c',
        'High-risk heterosexual contact (HRH)': '#d62728',
        'Heterosexual contact (Non-HRH)': '#9467bd',
        'Perinatal': '#8c564b',
        'Unknown risk': '#e377c2',
        'Other': '#7f7f7f'
    };
    
    // Create the legend
    function createLegend() {
        legendContainer.innerHTML = '';
        
        Object.entries(groupColors).forEach(([group, color]) => {
            const legendItem = document.createElement('div');
            legendItem.className = 'legend-item';
            
            const colorBox = document.createElement('div');
            colorBox.className = 'legend-color';
            colorBox.style.backgroundColor = color;
            
            const label = document.createElement('span');
            label.textContent = group;
            
            legendItem.appendChild(colorBox);
            legendItem.appendChild(label);
            legendContainer.appendChild(legendItem);
        });
    }
    
    // Filter data by population
    function filterByPopulation(population) {
        if (population === 'All') {
            return data;
        } else {
            return data.filter(item => item.Population === population);
        }
    }
    
    // Create radial visualization
    function createRadialVisualization(population, year = null) {
        const filteredData = filterByPopulation(population);
        
        // If a specific year is provided, filter by that year
        const yearData = year 
            ? filteredData.filter(item => item.Year === year)
            : filteredData;
            
        // Group data by transmission category
        const groupedData = {};
        yearData.forEach(item => {
            if (!groupedData[item.Group]) {
                groupedData[item.Group] = 0;
            }
            groupedData[item.Group] += item.Count;
        });
        
        // Prepare data for plotly
        const labels = Object.keys(groupedData);
        const values = Object.values(groupedData);
        const colors = labels.map(label => groupColors[label] || '#000000');
        
        // Create the pie chart with a hole in the middle
        const trace = {
            type: 'pie',
            labels: labels,
            values: values,
            marker: {
                colors: colors
            },
            textinfo: 'label+percent',
            textposition: 'outside',
            automargin: true,
            hole: 0.4,
            hoverinfo: 'label+value+percent',
            hoverlabel: {
                bgcolor: 'white',
                font: { size: 14 }
            }
        };
        
        const layout = {
            title: {
                text: `HIV Transmission Categories - ${population} Population${year ? ` (${year})` : ''}`,
                font: { size: 24 }
            },
            showlegend: false,
            height: 700,
            annotations: [{
                font: { size: 20 },
                showarrow: false,
                text: year ? `${year}` : 'All Years',
                x: 0.5,
                y: 0.5
            }]
        };
        
        Plotly.newPlot('visualization', [trace], layout);
    }
    
    // Create comparative visualization
    function createComparativeVisualization(population) {
        const filteredData = filterByPopulation(population);
        
        // Get unique years and groups
        const years = [...new Set(filteredData.map(item => item.Year))].sort();
        const groups = [...new Set(filteredData.map(item => item.Group))];
        
        // Create a trace for each transmission group
        const traces = [];
        
        groups.forEach(group => {
            const groupData = filteredData.filter(item => item.Group === group);
            const trace = {
                x: years,
                y: years.map(year => {
                    const yearData = groupData.find(item => item.Year === year);
                    return yearData ? yearData.Count : 0;
                }),
                type: 'scatter',
                mode: 'lines+markers',
                name: group,
                line: {
                    color: groupColors[group] || '#000000',
                    width: 3
                },
                marker: {
                    size: 8
                }
            };
            
            traces.push(trace);
        });
        
        const layout = {
            title: {
                text: `HIV Transmission Categories - ${population} Population (2011-2017)`,
                font: { size: 24 }
            },
            xaxis: {
                title: 'Year',
                tickvals: years
            },
            yaxis: {
                title: 'Number of Cases'
            },
            height: 700,
            hovermode: 'closest'
        };
        
        Plotly.newPlot('visualization', traces, layout);
    }
    
    // Animate through years
    function animateYears() {
        let yearIndex = 0;
        const years = [2011, 2012, 2013, 2014, 2015, 2016, 2017];
        
        playButton.disabled = true;
        playButton.textContent = 'Playing...';
        
        animationInterval = setInterval(() => {
            currentYear = years[yearIndex];
            createRadialVisualization(currentPopulation, currentYear);
            
            yearIndex++;
            if (yearIndex >= years.length) {
                clearInterval(animationInterval);
                playButton.disabled = false;
                playButton.textContent = 'Play Animation';
                
                // After animation completes, show all years
                setTimeout(() => {
                    createRadialVisualization(currentPopulation);
                }, 1000);
            }
        }, 1500); // Change every 1.5 seconds
    }
    
    // Initialize the visualization
    function init() {
        createTooltip();
        createLegend();
        createRadialVisualization('All');
        
        // Event listeners
        populationFilter.addEventListener('change', function() {
            currentPopulation = this.value;
            if (currentViewMode === 'radial') {
                createRadialVisualization(currentPopulation);
            } else {
                createComparativeVisualization(currentPopulation);
            }
        });
        
        viewMode.addEventListener('change', function() {
            currentViewMode = this.value;
            if (currentViewMode === 'radial') {
                createRadialVisualization(currentPopulation);
                playButton.disabled = false;
            } else {
                createComparativeVisualization(currentPopulation);
                playButton.disabled = true;
            }
        });
        
        playButton.addEventListener('click', function() {
            if (currentViewMode === 'radial') {
                animateYears();
            }
        });
    }
    
    // Run initialization when DOM is fully loaded
    document.addEventListener('DOMContentLoaded', init);
    </script>
</body>
</html>
"""

# Convert data to JSON for JavaScript
all_data = []
for _, row in grouped_df.iterrows():
    all_data.append({
        'Year': int(row['Year']),
        'Population': row['Population'],
        'Group': row['Group'],
        'Count': float(row['Count'])
    })

# Convert to JSON and insert into HTML
json_data = json.dumps(all_data)
html_content = html_content.replace('PLACEHOLDER_DATA', json_data)

# Write the HTML file
with open('../../interactive_viz_outputs/hiv_radial_visualization.html', 'w', encoding='utf-8') as f:
    f.write(html_content)

print("Radial visualization created as 'hiv_radial_visualization.html'")
print("You can open this file in any web browser")

Radial visualization created as 'hiv_radial_visualization.html'
You can open this file in any web browser
